In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os, sys
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

1. Explore data

In [ ]:
Sclinical_data=pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')
Sclinical_data.head(n=5)
      


In [ ]:
Tclinical_data=pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
print(Tclinical_data.head(5),'\n','='* 80)
peptides=pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv')
print(peptides.head(5) ,'\n','='* 80)
proteins=pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
print(proteins.head(5),'\n','='* 80)

In [ ]:
Sclinical_data.info()

In [ ]:
Tclinical_data.info()

In [ ]:
peptides.info()

In [ ]:
proteins.info()

In [ ]:

# merge dataframes
df_all = pd.merge(proteins, peptides[['visit_id', 'UniProt', 'Peptide','PeptideAbundance']], on=['visit_id', 'UniProt'], how='left')

# group by UniProt and calculate mean NPX
df_by_uniprot = df_all.groupby(['visit_id', 'UniProt']).agg({'NPX': 'mean'}).reset_index()

# group by Peptide and calculate mean PeptideAbundance
df_by_peptide = df_all.groupby(['visit_id', 'Peptide']).agg({'PeptideAbundance': 'mean'}).reset_index()

In [ ]:
df_by_uniprot.head()


In [ ]:
df_by_uniprot.shape

In [ ]:
df_by_peptide.head()

In [ ]:
df_by_peptide.shape

In [ ]:
df_uniprot = df_by_uniprot.pivot(index='visit_id',columns = 'UniProt', values = 'NPX').rename_axis(columns=None).reset_index()

In [ ]:
df_uniprot

In [ ]:
df_peptide = df_by_peptide.pivot(index='visit_id',columns = 'Peptide', values = 'PeptideAbundance').rename_axis(columns=None).reset_index()
df_peptide

In [ ]:
df_uniprot[['patient_id','visit_month']] = df_uniprot.visit_id.str.split("_", expand=True)
df_uniprot

In [ ]:
df_peptide[['patient_id','visit_month']] = df_peptide.visit_id.str.split("_", expand=True)
df_peptide

In [ ]:
df_merge= df_uniprot.merge(df_peptide, on=['patient_id','visit_month'],how='left')
df_merge

In [ ]:
Tclinical_new = Tclinical_data.melt(id_vars=['visit_id', 'patient_id', 
                                             'visit_month',
                                             'upd23b_clinical_state_on_medication'],
                                    var_name='updrs', value_name='rating')
Tclinical_new

In [ ]:
df_merge['visit_month']=df_merge['visit_month'].astype(int)
df_merge['patient_id']=df_merge['patient_id'].astype(int)
df_merge

In [ ]:
df_merge.columns

In [ ]:
df_train=df_merge.merge(Tclinical_new,on=['patient_id','visit_month'])
df_train

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train = (
    df_train
    .dropna(axis=0, subset=['rating'])
    .assign(
        updrs=df_train['updrs'].str.extract('(\d+)').astype(float),
        rating=df_train['rating'].astype(float),
    )
    .drop(['visit_id_x', 'visit_id'], axis=1)
)


In [ ]:
y = df_train.rating

In [ ]:
df_train

In [ ]:
# Select all columns except 'rating'
x = df_train.drop(columns=['rating'])

# Replace 'Off' and 'On' with 0 and 1, respectively
x = x.replace({'Off': 0, 'On': 1})

In [ ]:
x['upd23b_clinical_state_on_medication']=pd.to_numeric(x['upd23b_clinical_state_on_medication'])

In [ ]:
x.info()

In [ ]:
x.isnull().sum()

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(x, y, random_state=1)


impute=SimpleImputer()
imputed_X_train=pd.DataFrame(impute.fit_transform(train_X))
imputed_X_valid=pd.DataFrame(impute.transform(val_X))

imputed_X_train.columns= train_X.columns
imputed_X_valid.columns= val_X.columns

In [ ]:
train_X=train_X.fillna(0)
  

In [ ]:
amp_model=RandomForestRegressor(random_state=7)
imputed_X_train = imputed_X_train.fillna(0)
amp_model.fit(imputed_X_train, train_y)
imputed_X_valid = imputed_X_valid.fillna(0)
val_predictions = amp_model.predict(imputed_X_valid)

In [ ]:
val_predictions

In [ ]:
denominator = (val_y + np.abs(val_predictions)) / 200.0
diff = np.abs(val_y - val_predictions) / denominator
diff[denominator == 0] = 0.0
val_smape = np.mean(diff)

In [ ]:
val_smape

In [ ]:
val_y

In [ ]:
import amp_pd_peptide


In [ ]:
import amp_pd_peptide
env = amp_pd_peptide.make_env()
iter_test = env.iter_test()

counter = 0
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    predictions = process_test(test, test_proteins)
    submission = predictions[["prediction_id", "rating"]]
    env.predict(submission)
    
    if counter == 0:
        display(test)
        display(sample_submission)
        
    counter += 1

In [ ]:
pred=pd.DataFrame(['visit_id_x'],val_predictions)
pred
